In [1]:
import os
import pandas as pd
import numpy as np

np.random.seed(seed=42)

### 데이터 전처리

In [2]:
data_dir = "../data"
# preprocessed_output_dir = "../data/train"
raw_ratebeer = pd.read_json(os.path.join(data_dir, "raw_ratebeer.json"))

### 1. 결측치 제거 (label) reviewscore, beer_id NaN 제거

In [3]:
# reviewscore
print(raw_ratebeer.reviewscore.isna().sum())

# 인덱스 초기화
raw_ratebeer = raw_ratebeer[~raw_ratebeer.reviewscore.isna()]
raw_ratebeer.index = range(len(raw_ratebeer))

28


In [4]:
# beer_id
print(raw_ratebeer.beer_id.isna().sum())

# 인덱스 초기화
raw_ratebeer = raw_ratebeer[~raw_ratebeer.beer_id.isna()]
raw_ratebeer.index = range(len(raw_ratebeer))

779


### 2. user_id 등록

In [5]:
n_user = raw_ratebeer.profile_name.nunique()
print(f"n_user= {n_user:,}")

n_user= 60,580


In [6]:
profile_name2idx = {profile_name:user_id for user_id, profile_name in enumerate(raw_ratebeer.profile_name.unique())}
raw_ratebeer["user_id"] = [profile_name2idx[profile_name] for profile_name in raw_ratebeer.profile_name]

raw_ratebeer.head(1)

,reviewscore,reviewtime,reviewtext,appearance,aroma,palate,taste,overall,profile_name,beer_name,beer_id,brewer_id,abv,style,image_url,user_id
0,3.9,2022-05-15T17:26:51.803Z,Tap at Fermentoren CPH. Poured a hazy golden c...,3.0,8.0,4.0,8.0,16.0,jmgreenuk,Hill Farmstead Arthur,131594.0,11233.0,6.0,Saison / Farmhouse / Grisette,https://res.cloudinary.com/ratebeer/image/uplo...,0


### 3. 타입 변경

In [7]:
raw_ratebeer["beer_id"] = raw_ratebeer["beer_id"].astype(int)
raw_ratebeer["brewer_id"] = raw_ratebeer["brewer_id"].astype(int)
raw_ratebeer["user_id"] = raw_ratebeer["user_id"].astype(int)

### 4. Text 전처리

In [8]:
special_class1 = ["\\", "\cX", "\d", "\D", "\f", "\n", "\r", "\s", "\S", "\t", "\v", "\w", "\W", "\0", "[\b]"] # "\xhh", "\uhhhh", "\uhhhhh"
special_class2 = ["'", '"']

In [9]:
temp_text = raw_ratebeer["reviewtext"]

In [10]:
for special_str in special_class1:
    temp_text = temp_text.apply(lambda x: x.replace(special_str," "))
for special_str in special_class2:
    temp_text = temp_text.apply(lambda x: x.replace(special_str,""))

In [11]:
# meaningless (......)
temp_text = temp_text.apply(lambda x: x.replace(".....", ".."))
temp_text = temp_text.apply(lambda x: x.replace(".....", ".."))
temp_text = temp_text.apply(lambda x: x.replace(".....", ".."))

# special (delimeter)
temp_text = temp_text.apply(lambda x: x.replace("|"," "))

In [12]:
# double check
for i in special_class1:
    counts = 0  
    for texts in temp_text:
        if i in texts:
            counts +=1
        # print(texts)
        # break
    print(i.encode("unicode_escape"), counts, end=" | ") # fr"{i}", ":", 

for i in special_class2:
    counts = 0  
    for texts in temp_text:
        if i in texts:
            counts +=1
        # print(texts)
        # break
    print(i.encode("unicode_escape"), counts, end=" | ") # fr"{i}", ":", 

counts = 0  
for texts in temp_text:
    if "|" in texts:
        counts +=1
    # print(texts)
    # break
print(r"|", counts)


# 적용
raw_ratebeer["review_text"] = temp_text


b'\\\\' 0 | b'\\\\cX' 0 | b'\\\\d' 0 | b'\\\\D' 0 | b'\\x0c' 0 | b'\\n' 0 | b'\\r' 0 | b'\\\\s' 0 | b'\\\\S' 0 | b'\\t' 0 | b'\\x0b' 0 | b'\\\\w' 0 | b'\\\\W' 0 | b'\\x00' 0 | b'[\\x08]' 0 | b"'" 0 | b'"' 0 | | 0


In [13]:
## 저장
raw_ratebeer.to_csv(os.path.join(data_dir,"preprocessed_ratebeer.tsv"), sep = "\t")

## DB에 저장할 것

In [15]:
# target data

In [14]:
target_beer_id_list = [ 8666,  75661, 614833, 169964,    251,  10726,   5588,   5571,
                        703,   6668,   7656,   2228, 135361,    473,  36249,    709,
                        12775, 143730,    742,   4007,  72025,  58275, 221535, 291276,
                        2469,   2468, 553454,   1088,    721,  30913,    814,  75071,
                        715,   1267,     55,  17334,     37, 493355,  64253,  34662,
                        136165, 500620,  13833, 506847,  73158,    399,   5159,   3101,
                        315443, 552713, 911382, 906973,    268, 550283, 271470,    390,
                        71469,  35424,   4701,   2514,   2516, 288954,    405, 303473,
                        136785,    647, 632627,   3029,    717, 553245, 580686, 252000,
                        21936, 667837,   2811,    729,  71458, 634182,  86387,   1478,
                        146878, 146725, 149060,  15747, 718172,   3126, 480225, 268685,
                        730, 211962,  64518, 621308,   7580, 452525,   1417,   1486,
                        221]

In [15]:
df_target_beer_id = pd.DataFrame()
df_target_beer_id["beer_id"] = target_beer_id_list
df_target_beer_id.tail(3)

,beer_id
94,1417
95,1486
96,221


In [16]:
df_target_beer_id.to_csv(os.path.join(data_dir, "db_target_beer.csv"), index = False)

In [17]:
from datetime import datetime

#### DB-name : user ####
# user
user = raw_ratebeer.groupby('user_id')[['user_id', 'profile_name']].head(1).copy()
n_user = len(user)
user["password"] = [str(num).zfill(8) for num in np.random.randint(low = 0, high = 99999999, size = n_user)]
user["gender"] = "-"
user["birth"] = [datetime.strptime("1000-01-01 00:00:00.001", "%Y-%m-%d %H:%M:%S.%f")]*n_user
user = user[["user_id", "password", "profile_name", "gender", "birth"]]

print("n_user :", n_user)
user.head()

n_user : 60580


,user_id,password,profile_name,gender,birth
0,0,65682867,jmgreenuk,-,1000-01-01 00:00:00.001000
1,1,56755036,Ron,-,1000-01-01 00:00:00.001000
2,2,56882282,olberic,-,1000-01-01 00:00:00.001000
3,3,21081788,bergstaden,-,1000-01-01 00:00:00.001000
4,4,13315092,ZayZay,-,1000-01-01 00:00:00.001000


In [18]:
#### DB-name : beer ####
beer = raw_ratebeer.groupby("beer_name")[["beer_id","beer_name", "brewer_id", "abv", "style", "image_url"]].head(1).copy()
beer.index = range(len(beer))
print("n_beer :",  beer.shape[0])
beer.head()



n_beer : 9336


,beer_id,beer_name,brewer_id,abv,style,image_url
0,131594,Hill Farmstead Arthur,11233,6.0,Saison / Farmhouse / Grisette,https://res.cloudinary.com/ratebeer/image/uplo...
1,121536,O'Fallon Hemp Hop Rye,2399,5.5,Specialty Grain - Other,https://res.cloudinary.com/ratebeer/image/uplo...
2,12617,Hartwall Urho III,814,4.7,Pale Lager - American,https://res.cloudinary.com/ratebeer/image/uplo...
3,14940,Mendocino White Hawk Select IPA,65,7.0,IPA - English,https://res.cloudinary.com/ratebeer/image/uplo...
4,25933,Moorhouse's Blonde Witch (Cask),400,4.4,Blonde Ale / Golden Ale,https://res.cloudinary.com/ratebeer/image/uplo...


In [19]:
#### DB-name : review ####
review = raw_ratebeer[["user_id", "beer_id", "reviewscore", "review_text", "reviewtime", "appearance", "aroma", "palate", "taste", "overall"]].copy()
review.head()

,user_id,beer_id,reviewscore,review_text,reviewtime,appearance,aroma,palate,taste,overall
0,0,131594,3.9,Tap at Fermentoren CPH. Poured a hazy golden c...,2022-05-15T17:26:51.803Z,3.0,8.0,4.0,8.0,16.0
1,1,131594,3.9,Bottle I shared with JB. Beautiful cloudy str...,2022-05-04T19:05:02.673Z,4.0,8.0,4.0,8.0,15.0
2,2,131594,4.1,"2019 vintage Aroma is vinegar, sour apples, fr...",2021-10-24T12:35:44.710Z,4.0,8.0,4.0,9.0,16.0
3,3,131594,4.1,Golden with a white head - Malt aroma with som...,2021-08-31T20:31:34.537Z,3.0,8.0,4.0,9.0,17.0
4,4,131594,4.0,"Bouteille. Robe orangée, trouble, peu scintill...",2021-08-23T02:07:23.170Z,4.0,8.0,4.0,8.0,16.0


In [20]:
for i in range(129700, 129804):
    print(f"--{i}------------------------------------------------------------------------------------")
    print(review.iloc[i]["review_text"])

--129700------------------------------------------------------------------------------------
From a bottle. Pours a thick black with a lasting tan head, roasted coffee aroma, well carbonated, thick and rich with roasted malt and bitter chocolate/coffee flavours. I like it a lot, a real lot.
--129701------------------------------------------------------------------------------------
Couleur noire, très légère mousse qui disparait. Fort nez de malt grillé, café et chocolat noir. Gout doux de malt grillé, café et alcool. Corps assez aqueux
--129702------------------------------------------------------------------------------------
Bottle. Color brown with small white head. Aroma malt, rozsted, chocolate, coffee. Taste sweet, roasted, chocolate, alcohol, bourbon.
--129703------------------------------------------------------------------------------------
Draft at Tap 52. Brown pour with a kaki head. Light toasted malt aromas. Light toasted malts and mild black coffee notes. Average body an

In [21]:
user.to_csv(os.path.join(data_dir, "db_files/db_user.tsv"), index = False, sep = "\t")
beer.to_csv(os.path.join(data_dir, "db_files/db_beer.tsv"), index = False, sep = "\t")
review.to_csv(os.path.join(data_dir, "db_files/db_review.tsv"), index = False, sep ="\t")

In [24]:
## 학습용 데이터 만들기

### 4. 리뷰 개수가 10개 미만인 유저 제거(이거 보류)

In [22]:
all_ratebeer = raw_ratebeer.copy()

In [26]:
# # 리뷰 개수 10개 이상인 유저 리스트
# over10user_list = (train_ratebeer["user_id"].value_counts() >= 10).where(lambda x: x == True).dropna().index.tolist()

# # 리뷰 개수 10개 이상인 유저 리스트를 받아와서 해당 유저들로만 구성된 데이터셋으로 업데이트
# train_ratebeer = train_ratebeer[train_ratebeer["user_id"].isin(over10user_list)]

# # 인덱스 초기화
# train_ratebeer.index = range(len(train_ratebeer))

### 8. Train Test split

In [23]:
n_user = all_ratebeer.user_id.nunique()
test_ratio = 0.1
n_test_user = int(n_user*test_ratio)

np.random.seed(42)
test_user_idx_list = np.random.choice(range(n_user), size = n_test_user, replace=False)
test_user_list = all_ratebeer.user_id.unique()[test_user_idx_list]

train_ratebeer = all_ratebeer[~all_ratebeer["user_id"].isin(test_user_list)].drop("review_text", axis=1)
test_ratebeer = all_ratebeer[all_ratebeer["user_id"].isin(test_user_list)].drop("review_text", axis=1)

print(len(train_ratebeer)) # 2929083 
print(len(test_ratebeer)) # 314552

2929083
314552


In [24]:
# 저장
train_ratebeer.to_csv("../data/standard/train_ratebeer_without_text.csv", index = False)
test_ratebeer.to_csv("../data/standard/test_ratebeer_without_text.csv", index = False)

In [25]:
train_ratebeer['reviewscore'].unique()

array([3.9, 4.1, 3.8, 4. , 4.2, 3.7, 4.5, 4.8, 3.6, 4.7, 4.3, 4.9, 3.4,
       4.4, 5. , 3.5, 4.6, 3.3, 3. , 3.2, 2.9, 3.1, 2.5, 2.4, 2.7, 2.6,
       2.3, 2.8, 1.8, 1.9, 1.5, 1.6, 2.2, 2.1, 2. , 1.4, 1.7, 1.3, 0.9,
       1. , 0.5, 1.2, 1.1, 0.8, 0.6, 0.7])

In [38]:
train_ratebeer

,reviewscore,reviewtime,reviewtext,appearance,aroma,palate,taste,overall,profile_name,beer_name,beer_id,brewer_id,abv,style,image_url,user_id
0,3.9,2022-05-15T17:26:51.803Z,Tap at Fermentoren CPH. Poured a hazy golden c...,3.0,8.0,4.0,8.0,16.0,jmgreenuk,Hill Farmstead Arthur,131594,11233,6.0,Saison / Farmhouse / Grisette,https://res.cloudinary.com/ratebeer/image/uplo...,0
1,3.9,2022-05-04T19:05:02.673Z,Bottle I shared with JB. Beautiful cloudy str...,4.0,8.0,4.0,8.0,15.0,Ron,Hill Farmstead Arthur,131594,11233,6.0,Saison / Farmhouse / Grisette,https://res.cloudinary.com/ratebeer/image/uplo...,1
2,4.1,2021-10-24T12:35:44.710Z,"2019 vintage\nAroma is vinegar, sour apples, f...",4.0,8.0,4.0,9.0,16.0,olberic,Hill Farmstead Arthur,131594,11233,6.0,Saison / Farmhouse / Grisette,https://res.cloudinary.com/ratebeer/image/uplo...,2
3,4.1,2021-08-31T20:31:34.537Z,Golden with a white head - Malt aroma with som...,3.0,8.0,4.0,9.0,17.0,bergstaden,Hill Farmstead Arthur,131594,11233,6.0,Saison / Farmhouse / Grisette,https://res.cloudinary.com/ratebeer/image/uplo...,3
5,3.8,2021-06-05T18:28:43.450Z,"Tap at BeerOut, Forlì, Italy. Pours classic go...",3.0,8.0,4.0,8.0,15.0,rumore,Hill Farmstead Arthur,131594,11233,6.0,Saison / Farmhouse / Grisette,https://res.cloudinary.com/ratebeer/image/uplo...,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3243629,3.6,2002-03-10T20:17:53.000Z,"Smooth, malty and sweet, and a low hop presenc...",4.0,6.0,4.0,7.0,15.0,Dickinsonbeer,Appalachian Jolly Scot Scottish Ale,3324,574,5.2,Scottish Ale,https://res.cloudinary.com/ratebeer/image/uplo...,678
3243630,3.7,2001-12-28T01:50:14.000Z,"A surprisingly nice, smooth Scottish ale with ...",4.0,6.0,5.0,7.0,15.0,BillKismet,Appalachian Jolly Scot Scottish Ale,3324,574,5.2,Scottish Ale,https://res.cloudinary.com/ratebeer/image/uplo...,1945
3243631,3.0,2001-11-26T13:09:46.000Z,"Good beer, much beter on tap at the Brewery th...",3.0,5.0,3.0,6.0,13.0,bigmattiem,Appalachian Jolly Scot Scottish Ale,3324,574,5.2,Scottish Ale,https://res.cloudinary.com/ratebeer/image/uplo...,19946
3243633,2.9,2001-06-02T16:24:19.000Z,It's a sweet beer. I think the more I drank it...,3.0,5.0,3.0,7.0,11.0,jstu9,Appalachian Jolly Scot Scottish Ale,3324,574,5.2,Scottish Ale,https://res.cloudinary.com/ratebeer/image/uplo...,5307


In [166]:
########################################################################
########################################################################
########################################################################
########################################################################

## 데이터 셋 분리 [Testset 분리]
- 총 15000명 가량의 유저를 통해서 
- 한 2000명에 대해서 각 1

In [26]:
df_test = test_ratebeer.copy()

In [27]:
# 한 개를 맞출 수는 없다. 따라서 하나만 리뷰한 사람은 제외한다. (적어도 두 명의 유저가 있어야 하므로)
over1user_in_Test = (df_test.value_counts("user_id") == 1).where(lambda x: x == False).dropna().index.tolist()
df_test = df_test[df_test["user_id"].isin(over1user_in_Test)]
df_test.index = range(len(df_test))

In [28]:
dict_test_input = dict()
dict_test_answer = dict()

np.random.seed(42)
for _, (user_id, beer_id_list) in df_test.groupby("user_id")["beer_id"].apply(list).reset_index().iterrows():
    # 기본적으로 target beer들 이외의 맥주는 제외하려고 한다.
    # 하지만, 유저의 맥주에 대한 전반적인 선호도 파악도 잘하고 있는지 확인해야하므로, 
    # 타겟 맥주를 제외하고, 10%+1개의 맥주만을 남겨두고 제외한다.

    target_beers =  list(set(beer_id_list).intersection(target_beer_id_list))
    n_target_beers = len(target_beers)

    other_beers = list(set(beer_id_list)-set(target_beer_id_list))
    n_other_beers = len(other_beers)

    if n_other_beers > 0 : 
        n_add = int(n_other_beers*0.1) + 1 # 10%+1개의 맥주만을 남겨두고
        random_choice_idx = np.random.choice(range(n_other_beers), size = n_add, replace=False).tolist()
        other_beers_to_input = np.array(other_beers)[random_choice_idx].tolist()
    else:
        other_beers_to_input = []
    
    n_mask = min(int(n_target_beers*0.5), 8)
    random_choice_idx = np.random.choice(range(n_target_beers), size = n_mask, replace=False).tolist()

    target_beers_to_answer = np.array(target_beers)[random_choice_idx].tolist()
    target_beers_to_input = list(set(target_beers)-set(target_beers_to_answer))

    input_list = other_beers_to_input + target_beers_to_input
    answer_list = target_beers_to_answer

    # print(input_list)
    # print(answer_list)

    if input_list == [] or answer_list == []:
        continue


    dict_test_input[user_id] = input_list
    dict_test_answer[user_id] = answer_list
    
    # 유저별로 최대 8개를 정답지로 구성한다.
    # 기본적으로 한국맥주 중의 50%를 정답지로 마스킹해준다.
    # min(len())
    # 최대 8개 

In [29]:
df_test_input = pd.concat({k: pd.Series(v) for k, v in dict_test_input.items()})
df_test_answer = pd.concat({k: pd.Series(v) for k, v in dict_test_answer.items()})
#코드참고 https://stackoverflow.com/questions/42869544/dictionary-of-lists-to-dataframe

In [30]:
df_test_input = df_test_input.reset_index()
df_test_input.columns = ["user_id", "order", "beer_id"]
df_test_input = df_test_input.drop("order", axis=1)
df_test_input = df_test_input.merge(df_test, on=["user_id", "beer_id"])
print(len(df_test_input))
df_test_input.head(3)


33282


,user_id,beer_id,reviewscore,reviewtime,reviewtext,appearance,aroma,palate,taste,overall,profile_name,beer_name,brewer_id,abv,style,image_url
0,4,12027,3.8,2021-08-23T02:21:37.730Z,"Geuze fidèle au style en général. Brett belge,...",4.0,8.0,4.0,7.0,15.0,ZayZay,Boon Oude Geuze Mariage Parfait,748,8.0,Lambic - Gueuze,https://res.cloudinary.com/ratebeer/image/uplo...
1,4,59197,3.8,2020-10-24T01:01:04.880Z,"Robe ambré couleur abricot, trouble, sans reli...",4.0,8.0,4.0,8.0,14.0,ZayZay,Le Trou du Diable La Buteuse,6444,10.0,Tripel,https://res.cloudinary.com/ratebeer/image/uplo...
2,4,59877,3.5,2021-04-10T00:07:47.060Z,"Canette.\nRobe rosée, voilée, peu scintillante...",3.0,8.0,3.0,7.0,14.0,ZayZay,Dieu du Ciel! Rosée d'Hibiscus,364,5.9,Flavored - Other,https://res.cloudinary.com/ratebeer/image/uplo...


In [31]:
df_test_answer = df_test_answer.reset_index()
df_test_answer.columns = ["user_id", "order", "beer_id"]
df_test_answer = df_test_answer.drop("order", axis=1)
df_test_answer = df_test_answer.merge(df_test, on=["user_id", "beer_id"])
print(len(df_test_answer))
df_test_answer.head(3)

3631


,user_id,beer_id,reviewscore,reviewtime,reviewtext,appearance,aroma,palate,taste,overall,profile_name,beer_name,brewer_id,abv,style,image_url
0,4,37,1.9,2021-07-08T03:31:02.923Z,"Canette. \nRobe dorée, limpide, scintillante, ...",3.0,3.0,2.0,3.0,8.0,ZayZay,Heineken,9,5.0,Pale Lager - International / Premium,https://res.cloudinary.com/ratebeer/image/uplo...
1,4,473,1.7,2022-05-27T03:50:49.850Z,"robe dorée, translucide, limpide, effervescenc...",3.0,3.0,2.0,3.0,6.0,ZayZay,Budweiser,84,5.0,Pale Lager - American,https://res.cloudinary.com/ratebeer/image/uplo...
2,4,742,1.7,2020-10-24T21:25:51.827Z,"Robe jaune, limpide, scintillante, bulleuse.\n...",2.0,3.0,2.0,3.0,7.0,ZayZay,Corona Extra,119,4.5,Pale Lager - American,https://res.cloudinary.com/ratebeer/image/uplo...


In [32]:
df_test_input.to_csv("../data/test/test_input_ratebeer_without_text.csv", index = False)
df_test_answer.to_csv("../data/test/test_output_ratebeer_without_text.csv", index = False)

In [ ]:
########################################################################################################################################
########################################################################################################################################
########################################################################################################################################
########################################################################################################################################
########################################################################################################################################
########################################################################################################################################

### 8. training file making

In [33]:
raw_ratebeer[["user_id", "beer_id", "reviewscore", "reviewtime"]].isna().sum()

user_id        0
beer_id        0
reviewscore    0
reviewtime     0
dtype: int64

In [34]:
train_ratings = raw_ratebeer[["user_id", "beer_id", "reviewscore", "reviewtime"]]
train_ratings.columns = ["user", "item", "rating", "time"]
train_ratings.head(1)

,user,item,rating,time
0,0,131594,3.9,2022-05-15T17:26:51.803Z


In [35]:
train_ratings.to_csv(os.path.join(preprocessed_output_dir, "train_ratings.csv"), index=False)

NameError: name 'preprocessed_output_dir' is not defined

### 8. attribute extraction (style)

In [ ]:
style_df = raw_ratebeer.groupby("beer_id")["beer_id", "style"].head(1)
style_df.index = range(len(style_df))

In [ ]:
style_df

In [104]:
array, index = pd.factorize(style_df["style"])
style_df["style"] = array
style_df.groupby("beer_id")["style"].apply(list).to_json(os.path.join(preprocessed_output_dir, "rb_item2attributes.json"))